In [ ]:
from Problem import Problem
import networkx as nx
import matplotlib.pyplot as plt
from src.instance import Solution
from src.genetic_algorithm import GeneticAlgorithm
from itertools import product
import pandas as pd
import os

os.makedirs("experiments", exist_ok=True)
num_experiments = len(os.listdir("experiments"))

In [ ]:
number_cities = [100,1000]
alphas = [1,2]
betas = [1,2]
densities = [0.2,1]

In [ ]:
for i, (num_cities, alpha, beta, density) in enumerate(product(number_cities, alphas, betas, densities)):
    problem = Problem(num_cities, alpha=alpha, beta=beta, density=density)
    print(f"Testing with {num_cities} cities, alpha={alpha}, beta={beta}, density={density}")
    G = problem.graph
    pos_dict = {k: v['pos'] for k, v in G.nodes(data=True)}
    gold_dict = {n: data['gold'] for n, data in G.nodes(data=True)}
    paths_dict = nx.shortest_path(problem.graph, source=0, weight='dist')
    GA = GeneticAlgorithm(problem, max_generations= 100, mutation_rate=0.7, mutation_choice=0.5)
    best_solution, best_cost = GA.solve()
    base_cost = problem.baseline()
    print(f"Baseline solution: {base_cost}")
    print(f"GA solution: {best_cost}")
    log_dir = f"experiments/exp_{i+num_experiments}"
    os.makedirs(log_dir, exist_ok=True)
    GA.log(log_dir=log_dir)

In [ ]:
GA.plot_history()

In [ ]:
def plot_history(log_path="experiments/exp_0"):
    _, ax = plt.subplots()
    df_history = pd.read_csv(f"{log_path}/ga_history.csv")
    df_history['cumulative_average'] = df_history['best_cost'].expanding().mean()
    ax.plot(df_history['generation'], df_history['cumulative_average'], label='Cumulative Average of Best Cost')
    ax.plot(df_history['generation']-1, df_history['best_cost'], label='Best Cost per Generation', zorder=5)

    df_evaluations = pd.read_csv(f"{log_path}/ga_evaluations.csv")
    ax.scatter(df_evaluations['generation'], df_evaluations['evaluation_cost'], color='red', alpha=0.2, s=2, label='Offspring Evaluation')

    ax.set_xlabel('Generation')
    ax.set_ylabel('Cost')
    ax.legend()
    plt.show()

In [ ]:
plot_history(log_path="experiments/exp_0")